In [ ]:
def play(player1, player2):
    while player1.value > 0 and player2.value > 0:
        pass
        #player1.play(player2)

In [142]:
from random import choice
class Agent():
    def __init__(self, val=10):
        self.val = val
        self.rounds = 0
        self.history = []
        # TODO: add point values
    
    @property
    def value(self):
        return self.val
    @property
    def rounds_played(self):
        return self.rounds
    @property
    def view_history(self):
        return self.history
    
    def play(self, other):
        self.rounds += 1
        other.rounds += 1
        self_action = self.action()
        other_action = other.action()
        if self_action == 'cooperate' and other_action == 'cooperate':
            self.val += 1
            other.val += 1
        elif self_action == 'cheat' and other_action == 'cooperate':
            self.val += 3
            other.val -= 1
        elif self_action == 'cooperate' and other_action == 'cheat':
            self.val -= 1
            other.val += 3
        elif self_action == 'cheat' and other_action == 'cheat':
            self.val -= 2
            other.val -= 2

In [147]:
class Cooperator(Agent):
    def __init__(self):
        super().__init__()
        
    def action(self):
        act = 'cooperate'
        self.history.append(act)
        return act
    
class Cheater(Agent):
    def __init__(self):
        super().__init__()
        
    def action(self):
        act = 'cheat'
        self.history.append(act)
        return act
    
class Random(Agent):
    def __init__(self):
        super().__init__()
        
    def action(self):
        act = choice(['cheat', 'cooperate'])
        self.history.append(act)
        return act

In [148]:
g = Cooperator()
k = Cooperator()
l = Cheater()

In [149]:
for i in range(3):
    g.play(k)
g.value, k.value

(13, 13)

In [150]:
g.view_history

['cooperate', 'cooperate', 'cooperate']

In [151]:
g = Cooperator()
l = Cheater()
for i in range(3):
    g.play(l)
g.value, l.value

(7, 19)

In [152]:
g = Cooperator()
l = Cheater()
for i in range(3):
    l.play(g)
g.value, l.value

(7, 19)

In [154]:
g = Cooperator()
l = Random()
for i in range(3):
    l.play(g)
g.value, l.value, l.history

(11, 15, ['cooperate', 'cooperate', 'cheat'])